Ed - "I found a girrrl"

Me - *faints, collapses, rolls on the floor, foams in the mouth

In [2]:
import spotipy
import pandas as pd
import requests
import numpy as np
import time
from requests.exceptions import ReadTimeout
from spotipy.exceptions import SpotifyException
from timeit import default_timer as timer
from datetime import timedelta
import configparser
from spotipy.oauth2 import SpotifyClientCredentials
from pandas.api.types import CategoricalDtype
import configparser
import os

In [3]:
current_dir = os.path.dirname(os.path.abspath("Ed Sheeran"))

config_path = os.path.join(current_dir, '..', 'config.ini')

config = configparser.ConfigParser()

config.read(config_path)

client_id = config.get('credentials', 'Client_ID')
client_secret = config.get('credentials', 'Client_Secret')

client_credentials_manager = SpotifyClientCredentials(
                                client_id = client_id, client_secret = client_secret)

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager) 

In [4]:
def timekeeper(seconds):
    minutes = seconds // 60
    seconds = seconds % 60
    if minutes > 0:
        return f"{minutes} minutes, {seconds} seconds"
    else:
        return f"{seconds} seconds"

In [6]:
def collect_spotify_data():
    start_time = timer()
    data = []

    for i in range(0, 1000, 50):
        try:
            track_results = sp.search(q='artist:Ed Sheeran', type='track', limit=50, offset=i)
            for item in track_results['tracks']['items']:
                track_info = {
                    'artist_name': item['artists'][0]['name'],
                    'track_name': item['name'],
                    'track_id': item['id'],
                    'album_name': item['album']['name'],
                    'album_id': item['album']['id'],
                    'release_date': item['album']['release_date'],
                    'popularity': item['popularity'],
                    'explicit': item['explicit']
                }
                
                audio_features = sp.audio_features(item['id'])[0]
                if audio_features is not None:
                    audio_info = {
                        'danceability': audio_features.get('danceability', float('nan')),
                        'duration_ms': audio_features.get('duration_ms', float('nan')),
                        'energy': audio_features.get('energy', float('nan')),
                        'key': audio_features.get('key', float('nan')),
                        'loudness': audio_features.get('loudness', float('nan')),
                        'mode': audio_features.get('mode', float('nan')),
                        'speechiness': audio_features.get('speechiness', float('nan')),
                        'acousticness': audio_features.get('acousticness', float('nan')),
                        'instrumentalness': audio_features.get('instrumentalness', float('nan')),
                        'liveness': audio_features.get('liveness', float('nan')),
                        'valence': audio_features.get('valence', float('nan')),
                        'tempo': audio_features.get('tempo', float('nan')),
                        'time_signature': audio_features.get('time_signature', float('nan'))
                    }
                    track_info.update(audio_info)
                else:
                    audio_info = {
                        'danceability': float('nan'),
                        'duration_ms': float('nan'),
                        'energy': float('nan'),
                        'key': float('nan'),
                        'loudness': float('nan'),
                        'mode': float('nan'),
                        'speechiness': float('nan'),
                        'acousticness': float('nan'),
                        'instrumentalness': float('nan'),
                        'liveness': float('nan'),
                        'valence': float('nan'),
                        'tempo': float('nan'),
                        'time_signature': float('nan')
                    }
                    track_info.update(audio_info)

                
                if len(item['artists']) > 1:
                    featured_artists = [artist['name'] for artist in item['artists'][1:]]
                    track_info['featured_artists'] = featured_artists
                else:
                    track_info['featured_artists'] = []

                data.append(track_info)

        except requests.exceptions.ReadTimeout as e:
            print(f"Timeout error: {e}. Retrying in 5 seconds...")
            time.sleep(5) 
        except requests.exceptions.RequestException as e:
            print(f"API error: {e}")
            break 

    df_raw = pd.DataFrame(data)
    end_time = timer()
    elapsed_time = int(end_time - start_time)
    print(f"Elapsed time:", timekeeper(elapsed_time))
    return df_raw

df_spotify = collect_spotify_data()

Elapsed time: 4 minutes, 34 seconds


In [7]:
ed = df_spotify[df_spotify['artist_name'] == 'Ed Sheeran']
ed.to_csv('raw material.csv')

In [8]:
ed

,artist_name,track_name,track_id,album_name,album_id,release_date,popularity,explicit,danceability,duration_ms,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,featured_artists
0,Ed Sheeran,Shivers,50nfwKoDiSYg8zOCREWAm5,=,32iAEBstCjauDhyKpGjTuq,2021-10-25,83,False,0.788,207853,...,-2.724,1,0.0856,0.2810,0.000000,0.0424,0.822,141.020,4,[]
1,Ed Sheeran,Perfect,0tgVpDi06FyKpA1z0VMD4v,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,85,False,0.599,263400,...,-6.312,1,0.0232,0.1630,0.000000,0.1060,0.168,95.050,3,[]
2,Ed Sheeran,Shape of You,7qiZfU4dY1lWllzX7mPBI3,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,86,False,0.825,233713,...,-3.183,0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,4,[]
3,Ed Sheeran,Bad Habits,3rmo8F54jFF8OgYsqTxm5d,=,32iAEBstCjauDhyKpGjTuq,2021-10-25,80,False,0.807,230747,...,-3.745,0,0.0347,0.0451,0.000028,0.3660,0.537,126.011,4,[]
4,Ed Sheeran,Castle on the Hill,6PCUP3dWmTjcTtXY02oFdT,÷ (Deluxe),3T4tUhGYeRNVUGevb0wThu,2017-03-03,78,False,0.461,261154,...,-4.868,1,0.0989,0.0232,0.000011,0.1400,0.471,135.007,4,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,Ed Sheeran,Celestial,3tUBe0pZCvMO2RRDMNG4wR,Monday Motivation,1JxmXuQUB5jk3YVNfeGvTb,2023-11-17,2,False,0.574,209027,...,-1.248,1,0.0397,0.0529,0.000000,0.1610,0.501,123.038,4,[]
996,Ed Sheeran,Castle on the Hill - Acoustic,1Fxd5L6kNNTKfb9m0HM1Ep,Pop lento tranquilo,35sBkEPKe5GooLslN4ag2v,2024-02-11,3,False,0.563,226227,...,-5.857,1,0.0390,0.5200,0.000000,0.1460,0.617,145.561,4,[]
997,Ed Sheeran,Perfect,7vTm1hmRFgGogETWAdnGJQ,Cupid's Playlist,3U5Gjjp9Po7gTnZOLaIv1x,2024-01-26,1,False,0.598,263400,...,-6.312,1,0.0232,0.1620,0.000000,0.1060,0.168,95.009,3,[]
998,Ed Sheeran,Shape of You,3z4GBcrUpKPULmjQ5KCohw,Sucessos Internacionais,1DtRhb8Iek0iGv89pFn0wu,2023-07-14,1,False,0.825,233713,...,-3.183,0,0.0802,0.5810,0.000000,0.0931,0.931,95.977,4,[]


In [11]:
ed['album_name'].unique()

array(['=', '÷ (Deluxe)', 'x (Wembley Edition)',
       'No.6 Collaborations Project', 'x (Deluxe Edition)', '+',
       '- (Deluxe)', "I Don't Care (with Justin Bieber)",
       'Perfect Duet (Ed Sheeran & Beyoncé)',
       'Life Goes On (feat. Luke Combs)',
       'Beautiful People (feat. Khalid)',
       'Perfect Symphony (Ed Sheeran & Andrea Bocelli)',
       'x (10th Anniversary Edition)', 'Celestial',
       'BLOW (with Chris Stapleton & Bruno Mars)', 'Afterglow',
       'The Hobbit: The Desolation of Smaug (Original Motion Picture Soundtrack)',
       '2step (feat. Lil Baby)',
       'The Joker And The Queen (feat. Taylor Swift)', 'Merry Christmas',
       'Bad Habits (feat. Bring Me The Horizon)',
       'Bad Habits (MEDUZA Remix)', 'Shivers (Ofenbach Remix)',
       'Duets (30th Anniversary)', 'Autumn Variations',
       'Cross Me (feat. Chance the Rapper & PnB Rock)',
       '= (Tour Edition)', '+-=÷× (Tour Collection)', 'Spotify Singles',
       'South of the Border (feat. C

In [ ]:
studio_albums = [
    '=', 
    '÷ (Deluxe)', 
    'No.6 Collaborations Project', 
    'x (Deluxe Edition)', 
    '+',
    '- (Deluxe)',
    'Autumn Variations'
]

live_albums = [
    
    'x (Wembley Edition)',
]

remix_album = ['2step(The Remixes)']

compilation_album = [
    '+-=÷× (Tour Collection)', 'Spotify Singles'
    '=(Tour Edition)',
    '5'
]

ep = [
    'Loose Change',
    'No.5 Collaborations Project', 
    'Live at the Bedford',
    'Spotify Session'
]